In [21]:
import keras_tuner
import keras
from keras import backend as K
from tensorflow.keras.losses import mse
from keras.layers import Lambda, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [10]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    # by default, random_normal has mean=0 and std=1.0
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [11]:
def kl_divergence_multivariate_normal_keras(mu_p, log_var_p, mu_q, log_var_q):
    sigma_p = K.exp(log_var_p)            ## log variance to variance
    sigma_q = K.exp(log_var_q)

    term1 = K.sum(log_var_p - log_var_q)

    term2 = K.sum(sigma_q/sigma_p)

    term3 = K.sum(K.square(mu_q - mu_p) / sigma_p)

    return (term1 + term2 + term3)

In [12]:
def js_divergence_multivariate_normal_keras(mu_post, log_var_post, inputs_k, js_param):
    var_post = K.exp(log_var_post)            ## log variance to variance

    term1 = K.sum(log_var_post)
    term2 = K.sum(K.square(mu_post - inputs_k) / var_post)

    return js_param*(term1 + term2)

In [14]:
def vae_loss(alpha_js,inputs_k_pr, inputs_k,z_mean,z_log_var):
    js_param = (1.0-alpha_js)/alpha_js
    #reconstruction_loss = mse(inputs_x,outputs)
    kl_loss = (1.0/800.0)*kl_divergence_multivariate_normal_keras(inputs_k_pr, K.constant([0.0]), z_mean, z_log_var)
    label_loss = (1.0/800.0)*js_divergence_multivariate_normal_keras(z_mean,z_log_var,inputs_k,js_param)


    return  kl_loss + label_loss

In [15]:
# def model(hp):   
#     input_shape_k = (1, )
#     input_shape_x = (10,)
#     latent_dim = 1
    
#     inputs_k = Input(shape=input_shape_k, name='ground_truth')
#     inputs_k_pr = Input(shape=input_shape_k, name = 'prior mean')
#     # latent_inputs = Input(shape=(latent_dim,), name='z_sampling')
#     # intermediate_dim_1 = 8 # 1st layer
#     intermediate_dim_2 = 5 # 2nd Layer
#     intermediate_dim_3 = 3 # 3rd Layer


#     inputs_x = Input(shape=input_shape_x, name='temp_input')
#     inter_x1 = Dense(hp.Choice('units',[9,8,7,6]), activation='tanh', name='encoder_intermediate_1')(inputs_x)
#     inter_x2 = Dense(intermediate_dim_2, activation='tanh', name='encoder_intermediate_2')(inter_x1)
#     inter_x3 = Dense(intermediate_dim_3, activation='tanh', name='encoder_intermediate_3')(inter_x2)

# # q(z|x)
#     z_mean = Dense(latent_dim, name='z_mean')(inter_x3)
#     z_log_var = Dense(latent_dim, name='z_log_var')(inter_x3)

# # use reparameterization trick to push the sampling out as input
#     z = Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])
#     encoder = Model(inputs_x, [z_mean, z_log_var,z], name='encoder')
#     encoder.compile(optimizer = Adam(learning_rate= 0.001))
#     encoder.compile(loss = vae_loss(0.5,inputs_k_pr, inputs_k,z_mean,z_log_var))


#     return encoder


In [23]:
def build_model(hp):
  model = keras.Sequential()
  model.add(keras.layers.Dense(
      hp.Choice('units', [8, 16, 32]),
      activation='relu'))
  model.add(keras.layers.Dense(1, activation='relu'))
  model.compile(loss='mse')
  return model


In [27]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5)

Reloading Tuner from .\untitled_project\tuner0.json


In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
def generate_sin_samples(N = 1000, domain = 4):

    x = (np.random.rand(N, 1)-0.5) * domain * np.pi
    y = np.sin(x) + np.random.randn(N, 1) * 0.05

    return x, y

x,y = generate_sin_samples(N = 10000, domain = 4)

In [19]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.33, random_state=0)  

In [28]:


tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))
best_model = tuner.get_best_models()[0]